## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location,header = True, inferSchema = True)
df.show()

+----------+----+------+------+---+------+----+
total_bill| tip| sex|smoker|day| time|size|
+----------+----+------+------+---+------+----+
 16.99|1.01|Female| No|Sun|Dinner| 2|
 10.34|1.66| Male| No|Sun|Dinner| 3|
 21.01| 3.5| Male| No|Sun|Dinner| 3|
 23.68|3.31| Male| No|Sun|Dinner| 2|
 24.59|3.61|Female| No|Sun|Dinner| 4|
 25.29|4.71| Male| No|Sun|Dinner| 4|
 8.77| 2.0| Male| No|Sun|Dinner| 2|
 26.88|3.12| Male| No|Sun|Dinner| 4|
 15.04|1.96| Male| No|Sun|Dinner| 2|
 14.78|3.23| Male| No|Sun|Dinner| 2|
 10.27|1.71| Male| No|Sun|Dinner| 2|
 35.26| 5.0|Female| No|Sun|Dinner| 4|
 15.42|1.57| Male| No|Sun|Dinner| 2|
 18.43| 3.0| Male| No|Sun|Dinner| 4|
 14.83|3.02|Female| No|Sun|Dinner| 2|
 21.58|3.92| Male| No|Sun|Dinner| 2|
 10.33|1.67|Female| No|Sun|Dinner| 3|
 16.29|3.71| Male| No|Sun|Dinner| 3|
 16.97| 3.5|Female| No|Sun|Dinner| 3|
 20.65|3.35| Male| No|Sat|Dinner| 3|
+----------+----+------+------+---+------+----+
only showing top 20 rows

In [0]:
df.printSchema()

root
-- total_bill: double (nullable = true)
-- tip: double (nullable = true)
-- sex: string (nullable = true)
-- smoker: string (nullable = true)
-- day: string (nullable = true)
-- time: string (nullable = true)
-- size: integer (nullable = true)

In [0]:
df.columns

Out[35]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
### Handling Categorical Features
from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer(inputCols = ['sex','smoker','day','time'],
                        outputCols = ['sex_indexed','smoker_indexed','day_indexed','time_indexed'])
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
total_bill| tip| sex|smoker|day| time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
 16.99|1.01|Female| No|Sun|Dinner| 2| 1.0| 0.0| 1.0| 0.0|
 10.34|1.66| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|
 21.01| 3.5| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|
 23.68|3.31| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 24.59|3.61|Female| No|Sun|Dinner| 4| 1.0| 0.0| 1.0| 0.0|
 25.29|4.71| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|
 8.77| 2.0| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 26.88|3.12| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|
 15.04|1.96| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 14.78|3.23| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 10.27|1.71| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 35.26| 5.0|Female| No|Sun|Dinner| 4| 1.0| 0.0| 1.0| 0.0|
 15.42|1.57| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 18.43| 3.0| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|
 14.83|3.02|Female| No|Sun|Dinner| 2| 1.0| 0.0| 1.0| 0.0|
 21.58|3.92| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 10.33|1.67|Female| No|Sun|Dinner| 3| 1.0| 0.0| 1.0| 0.0|
 16.29|3.71| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|
 16.97| 3.5|Female| No|Sun|Dinner| 3| 1.0| 0.0| 1.0| 0.0|
 20.65|3.35| Male| No|Sat|Dinner| 3| 0.0| 0.0| 0.0| 0.0|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
only showing top 20 rows

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ['tip','size','sex_indexed','smoker_indexed','day_indexed','time_indexed'],
                outputCol = 'Independent Features')
output = featureassembler.transform(df_r)

In [0]:
output.select('Independent Features').show()

+--------------------+
Independent Features|
+--------------------+
[1.01,2.0,1.0,0.0...|
[1.66,3.0,0.0,0.0...|
[3.5,3.0,0.0,0.0,...|
[3.31,2.0,0.0,0.0...|
[3.61,4.0,1.0,0.0...|
[4.71,4.0,0.0,0.0...|
[2.0,2.0,0.0,0.0,...|
[3.12,4.0,0.0,0.0...|
[1.96,2.0,0.0,0.0...|
[3.23,2.0,0.0,0.0...|
[1.71,2.0,0.0,0.0...|
[5.0,4.0,1.0,0.0,...|
[1.57,2.0,0.0,0.0...|
[3.0,4.0,0.0,0.0,...|
[3.02,2.0,1.0,0.0...|
[3.92,2.0,0.0,0.0...|
[1.67,3.0,1.0,0.0...|
[3.71,3.0,0.0,0.0...|
[3.5,3.0,1.0,0.0,...|
(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows

In [0]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
total_bill| tip| sex|smoker|day| time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
 16.99|1.01|Female| No|Sun|Dinner| 2| 1.0| 0.0| 1.0| 0.0|[1.01,2.0,1.0,0.0...|
 10.34|1.66| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|[1.66,3.0,0.0,0.0...|
 21.01| 3.5| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|[3.5,3.0,0.0,0.0,...|
 23.68|3.31| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[3.31,2.0,0.0,0.0...|
 24.59|3.61|Female| No|Sun|Dinner| 4| 1.0| 0.0| 1.0| 0.0|[3.61,4.0,1.0,0.0...|
 25.29|4.71| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|[4.71,4.0,0.0,0.0...|
 8.77| 2.0| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[2.0,2.0,0.0,0.0,...|
 26.88|3.12| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|[3.12,4.0,0.0,0.0...|
 15.04|1.96| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[1.96,2.0,0.0,0.0...|
 14.78|3.23| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[3.23,2.0,0.0,0.0...|
 10.27|1.71| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[1.71,2.0,0.0,0.0...|
 35.26| 5.0|Female| No|Sun|Dinner| 4| 1.0| 0.0| 1.0| 0.0|[5.0,4.0,1.0,0.0,...|
 15.42|1.57| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[1.57,2.0,0.0,0.0...|
 18.43| 3.0| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|[3.0,4.0,0.0,0.0,...|
 14.83|3.02|Female| No|Sun|Dinner| 2| 1.0| 0.0| 1.0| 0.0|[3.02,2.0,1.0,0.0...|
 21.58|3.92| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[3.92,2.0,0.0,0.0...|
 10.33|1.67|Female| No|Sun|Dinner| 3| 1.0| 0.0| 1.0| 0.0|[1.67,3.0,1.0,0.0...|
 16.29|3.71| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|[3.71,3.0,0.0,0.0...|
 16.97| 3.5|Female| No|Sun|Dinner| 3| 1.0| 0.0| 1.0| 0.0|[3.5,3.0,1.0,0.0,...|
 20.65|3.35| Male| No|Sat|Dinner| 3| 0.0| 0.0| 0.0| 0.0|(6,[0,1],[3.35,3.0])|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
only showing top 20 rows

In [0]:
finalized_data = output.select('Independent Features','total_bill')
finalized_data.show()

+--------------------+----------+
Independent Features|total_bill|
+--------------------+----------+
[1.01,2.0,1.0,0.0...| 16.99|
[1.66,3.0,0.0,0.0...| 10.34|
[3.5,3.0,0.0,0.0,...| 21.01|
[3.31,2.0,0.0,0.0...| 23.68|
[3.61,4.0,1.0,0.0...| 24.59|
[4.71,4.0,0.0,0.0...| 25.29|
[2.0,2.0,0.0,0.0,...| 8.77|
[3.12,4.0,0.0,0.0...| 26.88|
[1.96,2.0,0.0,0.0...| 15.04|
[3.23,2.0,0.0,0.0...| 14.78|
[1.71,2.0,0.0,0.0...| 10.27|
[5.0,4.0,1.0,0.0,...| 35.26|
[1.57,2.0,0.0,0.0...| 15.42|
[3.0,4.0,0.0,0.0,...| 18.43|
[3.02,2.0,1.0,0.0...| 14.83|
[3.92,2.0,0.0,0.0...| 21.58|
[1.67,3.0,1.0,0.0...| 10.33|
[3.71,3.0,0.0,0.0...| 16.29|
[3.5,3.0,1.0,0.0,...| 16.97|
(6,[0,1],[3.35,3.0])| 20.65|
+--------------------+----------+
only showing top 20 rows

In [0]:
from pyspark.ml.regression import LinearRegression
### train test split
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
regressor = regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[43]: DenseVector([3.1872, 3.4452, -1.4803, 2.2227, -0.2434, -1.3995])

In [0]:
regressor.intercept

Out[44]: 1.8459235656776647

In [0]:
### Predictions
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+----------+------------------+
Independent Features|total_bill| prediction|
+--------------------+----------+------------------+
(6,[0,1],[1.97,2.0])| 12.02|15.015080607722343|
(6,[0,1],[2.01,2.0])| 20.23| 15.14256806600537|
 (6,[0,1],[3.0,4.0])| 20.45|25.188282380115865|
(6,[0,1],[3.15,3.0])| 20.08|22.221160487874442|
(6,[0,1],[3.18,2.0])| 19.82|18.871576220783936|
(6,[0,1],[3.39,2.0])| 11.61|19.540885376769833|
(6,[0,1],[4.08,2.0])| 17.92|21.740044032152063|
 (6,[0,1],[4.3,2.0])| 21.7|22.441225052708717|
(6,[0,1],[7.58,4.0])| 39.42| 39.78559635352256|
[1.25,2.0,1.0,0.0...| 8.51| 9.353680880786685|
[1.32,2.0,0.0,0.0...| 9.68|12.699995687427467|
[1.5,2.0,0.0,0.0,...| 19.08|11.630770997958194|
[1.5,2.0,0.0,0.0,...| 12.46|12.786861803309748|
[1.5,2.0,0.0,1.0,...| 11.59| 15.73980755341006|
[1.5,2.0,0.0,1.0,...| 15.69|15.496393830214387|
[1.5,2.0,1.0,0.0,...| 8.35|10.150477495055611|
[1.5,2.0,1.0,0.0,...| 10.65|10.150477495055611|
[1.56,2.0,0.0,0.0...| 9.94|13.464920437125635|
[1.64,2.0,0.0,1.0...| 15.36|16.186013657400657|
[1.76,2.0,0.0,1.0...| 11.24| 16.56847603224974|
+--------------------+----------+------------------+
only showing top 20 rows

In [0]:
pred_results.r2

Out[46]: 0.5545334742318232

In [0]:
pred_results.meanAbsoluteError

Out[47]: 3.9974125899018262

In [0]:
pred_results.meanSquaredError

Out[48]: 28.60098093290079